## 인터렉티브한 그래프를 활용하여 대시보드 만들기
---
서비스의 실적, 현황을 확인할 수 있는 대시보드를 작성하려고 합니다.

이용할 데이터셋과 라이브러리는 다음과 같습니다.

#### 데이터셋 (Bike Sharing Dataset)
UCI(http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset) 에서 제공하는 데이터셋입니다. 워싱턴DC의 자전거 공유서비스 업체 Capital bikeshare의 2011년~2012년 시간당 자전거 렌탈 실적과 날씨 정보가 포함되어있습니다.

#### 라이브러리
* 데이터분석 : ```pandas```,```numpy``` 
* 시각화 : ```hvplot```,```holoviews```
* 인터렉티브 대쉬보드 : ```panel```,```param```

#### 목표
이번 시각화의 대상은 시간당 자전거 대여(량)입니다. 년/월/일/시간의 변화에 따른 자전거 대여량을 손쉽게 확인할 수 있는 인터렉티브한 그래프를 만들겠습니다.

### 환경설정
필요한 패키지를 불러오고 기타 필요한 설정을 입력합니다.

In [ ]:
!pip install hvplot
!pip install panel

In [ ]:
# 데이터 분석를 위한 라이브러리
import pandas as pd
import numpy as np

# 컬럼이 최대 50개까지 잘리지 않고 보이게 됩니다
pd.options.display.max_columns = 50

# 데이터 시각화를 위한 패키지
import hvplot.pandas # matplotlib과 유사합니다
import holoviews as hv 
hv.extension('bokeh')
# 숫자의 천 단위 마다 콤마를 찍어주는 역할을 합니다
from bokeh.models.formatters import NumeralTickFormatter

# 인터렉티브 대시보드
import param
import panel as pn

# 한글 글꼴 설정
import matplotlib
matplotlib.rc('font', family='Malgun Gothic')
# %matplotlib inline

### 데이터셋 불러오기

In [ ]:
import os
print(os.listdir('../input'))

In [ ]:
hour_df = pd.read_csv('../input/bike-sharing-dataset/hour.csv')
print(hour_df.shape)
print(hour_df.info())

print("\nNull is hourly_demand columns: {0} => {1}".format(hour_df.columns.values, hour_df.isnull().sum().values))

hour_df.head(2)

해당 데이터는 결측치가 없기 때문에 전처리에 큰 노력은 필요 없어보입니다. 변수의 성격에 맞게 데이터 타입을 바꾸어 주거나, 새로운 변수를 생성하겠습니다.

**Data Fields**

* ```instant```: record index
* ```dteday```: date  
* ```season ```: 1 = spring, 2 = summer, 3 = fall, 4 = winter 
* ```yr```: year (0: 2011, 1:2012)
* ```mnth```: month ( 1 to 12)
* ```hr```:  hour (0 to 23)
* ```holiday```: weather day is holiday or not
* ```weekday```: day of the week
* ```workingday ```: if day is neither weekend nor holiday is 1, otherwise is 0.
* ```weather```
 * 1: Clear, Few clouds, Partly cloudy, Partly cloudy
 * 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist 
 * 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds 
 * 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
* ```temp```: Normalized temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-8, t_max=+39 (only in hourly scale)
* ```atemp```: Normalized feeling temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-16, t_max=+50 (only in hourly scale)
* ```hum```: Normalized humidity. The values are divided to 100 (max)
* ```windspeed```: Normalized wind speed. The values are divided to 67 (max)
* ```casual```: number of non-registered user rentals initiated
* ```registered```: number of registered user rentals initiated
* ```cnt```: number of total rentals

### 전처리

원활한 데이터 분석을 위해서 불필요한 변수 삭제, 변수 이름 정리, 데이터 타입 정리 등의 전처리를 하겠습니다.

In [ ]:
# 불필요한 컬럼 삭제
hour_df.drop(columns='instant', inplace=True)
# 컬럼 이름 정리
hour_df = hour_df.rename(columns={'dteday':'datetime', 'yr': 'year', 
                                  'mnth':'month', 'hr':'hour',
                                  'holiday':'is_holiday', 'workingday':'is_workingday',
                                  'weathersit':'weather_condition', 'hum':'humidity',
                                  'cnt':'count'})
####################
# 데이터 타입 정리 #  
####################

# 시간형식으로 변환
hour_df['datetime'] =  pd.to_datetime(hour_df['datetime'])
# 일자 컬럼 생성
hour_df['day'] =  pd.to_datetime(hour_df['datetime']).dt.day
# year컬럼은 읽기 쉽게 변경
hour_df.loc[hour_df['year'] == 0, 'year'] = 2011
hour_df.loc[hour_df['year'] == 1, 'year'] = 2012


# 카테고리 데이터형을 갖은 컬럼은 데이터타입을 바꾸어놓습니다.
hour_df['season'] = hour_df.season.astype('category')
hour_df['year'] = hour_df.year.astype('category')
hour_df['month'] = hour_df.month.astype('category')
hour_df['hour'] = hour_df.hour.astype('category')
hour_df['day'] = hour_df.day.astype('category')
hour_df['is_holiday'] = hour_df.is_holiday.astype('category')
hour_df['weekday'] = hour_df.weekday.astype('category')
hour_df['is_workingday'] = hour_df.is_workingday.astype('category')
hour_df['weather_condition'] = hour_df.weather_condition.astype('category')

# make it tidy
hour_df.drop(columns='count', inplace=True)
hour_df = hour_df.melt(id_vars=['datetime', 'season', 'year', 'month', 'hour', 
                                'is_holiday', 'weekday','is_workingday', 'weather_condition', 
                                'temp', 'atemp', 'humidity', 'windspeed','day'],
                       var_name='rental_type', value_name='count')

hour_df['rental_type'] = hour_df.rental_type.astype('category')

# 미리보기
print(hour_df.info())
hour_df.head(2)

### EDA (탐색적 데이터 분석)

시간 변화에 따른 실적(자전거 대여량)을 어떻게 나타내면 좋을지 아이디어를 얻기위해서 가지고 있는 데이터를 둘러보겠습니다

In [ ]:
# 연도별 렌탈타입에 따른 대여량 비교 (비회원 vs 회원)
hour_df[['year','rental_type','count']].\
groupby(['year','rental_type']).\
aggregate(np.sum).\
hvplot.bar(x='year', y='count', by='rental_type', 
           yformatter=NumeralTickFormatter(), title='연도별 렌탈타입에 따른 대여량 비교 (비회원 vs 회원)')

In [ ]:
# 전년대비 성장률
table = hour_df[['year','rental_type','count']].groupby(['year','rental_type']).aggregate(np.sum).T
table[2012]/table[2011]

In [ ]:
# 렌탈타입에 따른 월별  대여량
year_2011 = hour_df[hour_df.year == 2011][['month','rental_type','count']].\
            groupby(['month', 'rental_type']).\
            aggregate(np.sum).\
            hvplot.bar(x='month', y='count', by='rental_type', stacked=True,
                       yformatter=NumeralTickFormatter(),
                       title='렌탈타입에 따른 월별 대여량(2011)',
                       legend='top_left')

year_2012 = hour_df[hour_df.year == 2012][['month','rental_type','count']].\
            groupby(['month', 'rental_type']).\
            aggregate(np.sum).\
            hvplot.bar(x='month', y='count', by='rental_type', stacked=True,
                       yformatter=NumeralTickFormatter(),
                       title='렌탈타입에 따른 월별  대여량(2012)',
                       legend='top_left')

layout = year_2011+year_2012
layout.cols(1)

In [ ]:
# 계절에 따른 시간별 평균 대여량
# 1 = spring, 2 = summer, 3 = fall, 4 = winter 
year_2011 = hour_df[hour_df.year == 2011][['hour','season','count']].\
            groupby(['season','hour']).\
            aggregate(np.mean).\
            hvplot(x='hour', y='count', by='season', legend='top_left', 
                   title='계절에 따른 시간별 평균 대여량(2011)')

year_2012 = hour_df[hour_df.year == 2012][['hour','season','count']].\
            groupby(['season','hour']).\
            aggregate(np.mean).\
            hvplot(x='hour', y='count', by='season', legend='top_left', 
                   title='계절에 따른 시간별 평균 대여량(2012)')

layout = year_2011+year_2012
layout.cols(1)

In [ ]:
year_2011 = hour_df[hour_df.year == 2011][['hour','weekday','count']].\
            groupby(['weekday','hour']).\
            aggregate(np.mean).\
            hvplot(x='hour', y='count', by='weekday', legend='top_left', 
                   title='요일에 따른 시간별 평균 대여량(2011)')

year_2012 = hour_df[hour_df.year == 2012][['hour','weekday','count']].\
            groupby(['weekday','hour']).\
            aggregate(np.mean).\
            hvplot(x='hour', y='count', by='weekday', legend='top_left', 
                   title='요일에 따른 시간별 평균 대여량(2012)')

layout = year_2011+year_2012
layout.cols(1)

In [ ]:
# 계절 평균 대여량
hour_df[['year','season','count']].\
groupby(['year','season']).\
aggregate(np.mean).\
hvplot.step(x='season', y='count', by='year', title='계절 평균 대여량', yformatter="%d", xticks=[1,2,3,4], padding=(.1,.1))

In [ ]:
# 시간별 대여량 아웃라이어 확인
spring = hour_df[hour_df.season == 1][['hour','count']].\
         hvplot.box(y='count', by='hour', title='시간별 대여량의 이상치(봄)')

summer = hour_df[hour_df.season == 2][['hour','count']].\
         hvplot.box(y='count', by='hour', title='시간별 대여량의 이상치(여름)')

fall = hour_df[hour_df.season == 3][['hour','count']].\
         hvplot.box(y='count', by='hour', title='시간별 대여량의 이상치(가을)')

winter = hour_df[hour_df.season == 4][['hour','count']].\
         hvplot.box(y='count', by='hour', title='시간별 대여량의 이상치(겨울)')

layout = spring + summer + fall + winter
layout.cols(1)

### 1. 시간에 따른 자전거 대여량을 구하는 그래프를 출력하기
사용자가 입력한 조건에 따라 실적을 확인할 수 있는 그래프를 만들겠습니다.

In [ ]:
# 월별 대여량
class BikeExplorer_monthly(param.Parameterized):
    year = param.Integer(default=2012, bounds=(2011,2012))
    rental_type = param.ObjectSelector(default='All', objects=['All','Sorted'])
 
    def make_view(self, **kwargs):  
        # 이용자 타입에 따라 다른 그래프를 출력합니다
        if self.rental_type == 'All':
            df = hour_df[['year','month','count']]
            df = df[(df.year == self.year)].groupby(by=['month']).aggregate(np.sum)
         
            return df.hvplot.bar(x='month',y='count', 
                             yformatter=NumeralTickFormatter(),legend='top_left',
                             title='월별 대여량 (전체 이용자)')  
        else:            
            df = hour_df[['year','month','rental_type','count']]
            df = df[(df.year == self.year)].groupby(by=['rental_type','month']).aggregate(np.sum)
            
            return df.hvplot.bar(x='month',y='count',by='rental_type', stacked=True,
                             yformatter=NumeralTickFormatter(),legend='top_left',
                             title='월별 대여량 (유저별)')

In [ ]:
# 실제 작동과는 무관한 오류 메시지 입니다.
explorer_monthly = BikeExplorer_monthly()
r = pn.Row(explorer_monthly, explorer_monthly.make_view)
r

In [ ]:
# 일별 대여량
class BikeExplorer_daily(param.Parameterized):
    year = param.Integer(default=2012, bounds=(2011,2012))
    month = param.Integer(default=1, bounds=(1,12))
    rental_type = param.ObjectSelector(default='All', objects=['All','Sorted'])
    
    def make_view(self, **kwargs):  
        # 이용자 타입에 따라 다른 그래프를 출력합니다
        if self.rental_type == 'All':
            df = hour_df[['year','month','day','count']]
            df = df[(df.year == self.year) & (df.month == self.month)].\
                                    groupby(by=['day']).aggregate(np.sum)
         
            return df.hvplot.bar(x='day',y='count',
                             yformatter=NumeralTickFormatter(),legend='top_left',
                             title='일별 대여량 (전체 이용자)')  
        else:            
            df = hour_df[['year','month','day','rental_type','count']]
            df = df[(df.year == self.year) & (df.month == self.month)].\
                                    groupby(by=['rental_type','day']).aggregate(np.sum)
            
            return df.hvplot.bar(x='day',y='count', by='rental_type', stacked=True,
                             yformatter=NumeralTickFormatter(),legend='top_left',
                             title='일별 대여량 (유저별)')

In [ ]:
# 실제 작동과는 무관한 오류 메시지 입니다.
explorer_daily = BikeExplorer_daily()
r = pn.Row(explorer_daily, explorer_daily.make_view)
r

In [ ]:
# 시간별 대여량
class BikeExplorer_hourly(param.Parameterized):
    year = param.Integer(default=2012, bounds=(2011,2012))
    month = param.Integer(default=1, bounds=(1,12))
    day = param.Integer(default=1, bounds=(1,31))
    rental_type = param.ObjectSelector(default='All', objects=['All','Sorted'])
    
    def make_view(self, **kwargs):  
        # 이용자 타입에 따라 다른 그래프를 출력합니다
        if self.rental_type == 'All':
            df = hour_df[['year','month','day','hour','count']]
            df = df[(df.year == self.year) & (df.month == self.month) & (df.day == self.day)].\
                                    groupby(by=['hour']).aggregate(np.sum)
         
            return df.hvplot.bar(x='hour',y='count',
                             yformatter=NumeralTickFormatter(),legend='top_left',
                             title='시간별 대여량 (전체 이용자)')  
        else:            
            df = hour_df[['year','month','day','hour','rental_type','count']]
            df = df[(df.year == self.year) & (df.month == self.month) & (df.day == self.day)].\
                                    groupby(by=['rental_type','hour']).aggregate(np.sum)
            
            return df.hvplot.bar(x='hour',y='count', by='rental_type', stacked=True,
                             yformatter=NumeralTickFormatter(),legend='top_left',
                             title='시간별 대여량 (유저별)')

In [ ]:
# 실제 작동과는 무관한 오류 메시지 입니다.
explorer_hourly = BikeExplorer_hourly()
r = pn.Row(explorer_hourly, explorer_hourly.make_view)
r

### 마치며
서비스를 운영중인 회사라면 서비스의 현황을 파악하는 것이 매우 중요하다고 생각합니다. 현황을 파악할 수 있어야 회사의 현 상태를 점검할 수 있고, 문제를 발견하여 빠르게 개선할 수 있기 때문입니다. 예를들어 당월 매출이 목표치를 달성하지 못할 것 같다면, 영업팀이나 마케팅팀은 목표 달성을 위한 다른 방법들을 강구해 볼 것입니다.

이런 점에서 생각해 볼 때, 무엇을 관측대상으로 삼을 것인지 결정하는 일은 매우 중요합니다. 추적관리하는 지표가 무엇이냐에 따라 다른 의사결정을 할 수 밖에 없기 때문입니다. 그리고 이부분에서 회사의, 혹은 담당자의 능력이 큰 영향을 미친다고 생각합니다.

아쉽게도 이번 데이터셋에서는 대시보드 위에 나타낼만한 내용이 적어서 이만 마칩니다.